In [1]:
import mo_descriptor as md
import nn_frame as nn
import numpy as np

2022-07-08 01:15:22.873497: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-08 01:15:22.873618: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
'''
prepare data_set
1. make mo_pair descriptor
'''
# x_shift = np.arange(0, 4.1, 0.1)
# y_shift = np.arange(0, 4.1, 0.1)
# z_shift = np.zeros(x_shift.shape)
# # the original mo, e.g. homo
# homo = md.MO_descriptor('data/homo-s0.cube').make()
# lumo = md.MO_descriptor('data/lumo-s0.cube').make()

# # for the original pair of one mo and itself
# homo_pair = md.MO_pair_descriptor(homo, homo).make()
# lumo_pair = md.MO_pair_descriptor(lumo, lumo).make()

# # for the shifted pair
# homo_pairs = np.zeros((len(x_shift)*len(y_shift),) + homo_pair.shape)
# lumo_pairs = np.zeros((len(x_shift)*len(y_shift),) + lumo_pair.shape)
# for ii, i in enumerate(x_shift):
#     for jj, j in enumerate(y_shift):
#         idx = ii * len(y_shift) + jj
#         homo_pairs[idx][0] = homo_pair[0]
#         homo_pairs[idx][1] = np.add(homo_pair[1],i)
#         homo_pairs[idx][2] = np.add(homo_pair[2],j)
#         homo_pairs[idx][3] = homo_pair[3]
#         lumo_pairs[idx][0] = lumo_pair[0]
#         lumo_pairs[idx][1] = np.add(lumo_pair[1],i)
#         lumo_pairs[idx][2] = np.add(lumo_pair[2],j)
#         lumo_pairs[idx][3] = lumo_pair[3]
# np.save('homo_homo_pair.npy', homo_pairs)
# np.save('lumo_lumo_pair.npy', lumo_pairs)
homo_pairs = np.load('homo_homo_pair.npy')
lumo_pairs = np.load('lumo_lumo_pair.npy')

In [3]:
'''
2. read coupling
'''
raw_data = np.loadtxt('data/cdft-V1V2.dat')
c_homo = np.add(raw_data[:,2], raw_data[:,3]) * 1/2
c_lumo = np.add(raw_data[:,4], raw_data[:,5]) * 1/2

In [4]:
train_homo_pairs = homo_pairs[0:1200]
train_lumo_pairs = lumo_pairs[0:1200]

train_c_homo = c_homo[0:1200]
train_c_lumo = c_lumo[0:1200]

test_homo_pairs = homo_pairs[1200:]
test_lumo_pairs = lumo_pairs[1200:]

test_c_homo = c_homo[1200:]
test_c_lumo = c_lumo[1200:]

In [5]:
NN = nn.NN()
NN.train(train_homo_pairs,train_c_homo)
NN.test(test_homo_pairs, test_c_homo)

No setting is specified, default setting will be applied.


2022-07-08 01:15:24.519850: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-08 01:15:24.519985: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Deng-PC): /proc/driver/nvidia/version does not exist
2022-07-08 01:15:24.520385: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TypeError: in user code:

    File "/mnt/e/work/ML_MO_COUPLE/nn_frame.py", line 80, in train_step  *
        self.loss = self.loss_function(Y, predictions)
    File "/mnt/e/work/ML_MO_COUPLE/nn_frame.py", line 74, in loss_function  *
        return tf.keras.losses.MeanSquaredError(true, predict) # MSE loss function, only for test
    File "/home/dgg95223/venv/lib/python3.7/site-packages/keras/losses.py", line 310, in __init__  **
        super().__init__(mean_squared_error, name=name, reduction=reduction)
    File "/home/dgg95223/venv/lib/python3.7/site-packages/keras/losses.py", line 227, in __init__
        super().__init__(reduction=reduction, name=name)
    File "/home/dgg95223/venv/lib/python3.7/site-packages/keras/losses.py", line 88, in __init__
        losses_utils.ReductionV2.validate(reduction)
    File "/home/dgg95223/venv/lib/python3.7/site-packages/keras/utils/losses_utils.py", line 82, in validate
        if key not in cls.all():

    TypeError: Expected float64, but got 